In [1]:
# coding: utf-8
import pandas as pd

train = pd.read_csv("train.csv")
target = pd.read_csv("test_public.csv")

In [2]:
# encode label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['subject'])
train.loc[:,'subject_no'] = pd.Series(le.transform(train['subject']), index=train.index)
train.head()

,content_id,content,subject,sentiment_value,sentiment_word,subject_no
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响,0
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高,0
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低,0
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性,0
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高,0


In [11]:
def clean_dump(train_df):
    train_dict = {}
    for ind, row in train_df.iterrows():
        content, label = row['content'], row['label']
        if train_dict.get(content) is None:
            train_dict[content] = {label}
        else:
            train_dict[content].add(label)
    
    conts = []
    labels = []
    for k, v in train_dict.items():
        conts.append(k)
        labels.append(v)
    return conts, labels

In [28]:
train['label'] = train['subject'].str.cat(train['sentiment_value'].astype(str))
train_contents, train_labels = clean_dump(train)
train_labels[38]

{'价格1', '配置1'}

In [30]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
subject_onehot = mlb.fit_transform(train_labels)
subject_onehot[38]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1])

In [5]:
import jieba
content_col = train['content'].apply(lambda x: [word for word in jieba.cut(x)])
content_col_tar = target['content'].apply(lambda x: [word for word in jieba.cut(x)])
train.loc[:, 'content_words'] = pd.Series([" ".join(words) for words in content_col], index=train.index)
target.loc[:, 'content_words'] = pd.Series([" ".join(words) for words in content_col_tar], index=target.index)
# content shape
print(len(content_col), max([len(x) for x in content_col]))
print(len(content_col_tar), max([len(x) for x in content_col_tar]))
max_length=max([len(x) for x in content_col])


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.616 seconds.
Prefix dict has been built succesfully.


9947 128
2364 127


In [6]:
vocab = set()
for words in content_col.values:
    for word in words:
        vocab.add(word)
for words in content_col_tar.values:
    for word in words:
        vocab.add(word)
vocab_size = len(vocab)

In [9]:
from keras.preprocessing.text import one_hot
encoded_train = [one_hot(d, vocab_size) for d in train['content_words'].values]
encoded_target = [one_hot(d, vocab_size) for d in target['content_words'].values]

In [ ]:
# In[8]:


from keras.preprocessing.sequence import pad_sequences
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_target = pad_sequences(encoded_target, maxlen=max_length, padding='post')


# In[9]:


from keras.models import Sequential,Model
from keras.layers import Flatten, Dense, Embedding, Input
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.layers import Input, Dense, Masking, LSTM, Bidirectional

OUTPUT_SIZE = 10
CELL_SIZE = 50
BATCH_SIZE = 32
LR = 0.001
epoche = 100 # 训练100轮

# define the model 定义模型
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(CELL_SIZE, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(OUTPUT_SIZE, activation='softmax'))
# optimizer
model.compile(optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy'])

# summarize the model 打印模型信息
print(model.summary())


# In[10]:


from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

X_train, y_train = padded_train, subject_onehot
# Fit the model
model.fit(X_train, y_train,
  epochs=epoche,
  batch_size=BATCH_SIZE,
  validation_split=0.2,
  verbose=1, callbacks=callbacks_list,
          class_weight= class_weights
  )


# In[12]:


doc_predicted = model.predict(padded_target)
prdc= enc.inverse_transform(doc_predicted).reshape(len(content_col_tar))


# In[13]:


submit = pd.DataFrame(columns=['content_id'])
submit['content_id']=target['content_id']
submit['subject'] = pd.Series(le.inverse_transform(prdc), index=target.index)
submit['sentiment_value'] = 0
submit['sentiment_word'] = ""
submit.to_csv("submit.csv", index=False)


# In[11]:




